In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import pandas as pd
from sklearn import dummy, metrics, neighbors, svm
from sklearn.model_selection import train_test_split

# Introdução ao Aprendizado de Máquina

## Trabalhando com dados

### Carregando os datasets

Nós iremos duas vertentes do mesmo dataset, com propósitos diferentes. `Weather` é um dataset educacional (composto por dados fictícios) amplamente usado para ensinar conceitos de aprendizado de máquina. Ele contém 14 instâncias, 4 features e é considerado um dataset de classificação binária.

In [3]:
weather_nominal = pd.read_csv('./weather.nominal.csv')

weather_numeric = pd.read_csv('./weather.csv')

weather_nominal
weather_numeric

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,sunny,mild,high,False,no
8,sunny,cool,normal,False,yes
9,rainy,mild,normal,False,yes


,outlook,temperature,humidity,windy,play
0,sunny,85,85,False,no
1,sunny,80,90,True,no
2,overcast,83,86,False,yes
3,rainy,70,96,False,yes
4,rainy,68,80,False,yes
5,rainy,65,70,True,no
6,overcast,64,65,True,yes
7,sunny,72,95,False,no
8,sunny,69,70,False,yes
9,rainy,75,80,False,yes


Aqui vamos conferir os possíveis valores únicos em colunas categóricas:

In [4]:
print('Outlook:', weather_nominal.outlook.unique())
print('Temperature:', weather_nominal.temperature.unique())
print('Humidity:', weather_nominal.humidity.unique())
print('Windy:', weather_nominal.windy.unique())
print('Play:', weather_nominal.play.unique())

Outlook: ['sunny' 'overcast' 'rainy']
Temperature: ['hot' 'mild' 'cool']
Humidity: ['high' 'normal']
Windy: [False  True]
Play: ['no' 'yes']


Mostrando dataset com `head` e `tail` (bom para datasets extensos).

In [5]:
weather_nominal.head()
weather_numeric.tail()

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


,outlook,temperature,humidity,windy,play
9,rainy,75,80,False,yes
10,sunny,75,70,True,yes
11,overcast,72,90,True,yes
12,overcast,81,75,False,yes
13,rainy,71,91,True,no


### Transformando os dados categóricos em numéricos

Para usar os datasets, nós devemos transformar os dados categóricos em valores numéricos, já que tanto nosso método de classificação quanto o de regressão trabalham apenas com números. Iremos usar uma técnica chamada `One-hot enconde`, que gera uma matriz esparsa afirmando qual valor está presente naquela posição.

In [12]:
# Mudando o tipo das colunas categóricas
weather_numeric['outlook'] = weather_numeric['outlook'].astype('category')
weather_numeric['windy'] = weather_numeric['windy'].astype('category')
weather_numeric['play'] = weather_numeric['play'].astype('category')

# Aplicando enconding às colunas selecionadas
cat_columns = weather_numeric.select_dtypes(['category']).columns
weather_numeric[cat_columns] = weather_numeric[cat_columns].apply(lambda x: x.cat.codes)
weather_numeric.head()

,outlook,temperature,humidity,windy,play
0,2,85,85,0,0
1,2,80,90,1,0
2,0,83,86,0,1
3,1,70,96,0,1
4,1,68,80,0,1


### Dividindo datasets em treino e teste

In [ ]:
X_nominal_train, X_nominal_test, y_nominal_train, y_nominal_test = train_test_split(
    weather_nominal.iloc[:, :-1], 
    weather_nominal.iloc[:, -1], 
    test_size=0.3
)

X_nominal_train
X_nominal_test
y_nominal_train
y_nominal_test

In [13]:
X_numeric_train, X_numeric_test, y_numeric_train, y_numeric_test = train_test_split(
    weather_numeric.iloc[:, :-1], 
    weather_numeric.iloc[:, -1], 
    test_size=0.3
)

X_numeric_train
X_numeric_test
y_numeric_train
y_numeric_test

,outlook,temperature,humidity,windy
1,2,80,90,1
12,0,81,75,0
5,1,65,70,1
4,1,68,80,0
2,0,83,86,0
3,1,70,96,0
11,0,72,90,1
0,2,85,85,0
13,1,71,91,1


,outlook,temperature,humidity,windy
7,2,72,95,0
8,2,69,70,0
9,1,75,80,0
10,2,75,70,1
6,0,64,65,1


1     0
12    1
5     0
4     1
2     1
3     1
11    1
0     0
13    0
Name: play, dtype: int8

7     0
8     1
9     1
10    1
6     1
Name: play, dtype: int8

## Modelos

### Classificação

Usando o KNN para classificar as instâncias categóricas do `Weather`.

In [14]:
classification = neighbors.KNeighborsClassifier(n_neighbors=3)
classification.fit(X_numeric_train, y_numeric_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [20]:
class_predict = classification.predict(X_numeric_test)
class_predict

array([1, 1, 1, 1, 1], dtype=int8)

### Regressão

Usando o SVM para fazer a regressão das instâncias numéricas do `Weather`.

In [16]:
regression = svm.LinearSVR()
regression.fit(X_numeric_train, y_numeric_train)

/home/ingrid/.local/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


LinearSVR(C=1.0, dual=True, epsilon=0.0, fit_intercept=True,
     intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
     random_state=None, tol=0.0001, verbose=0)

In [19]:
regr_predict = regression.predict(X_numeric_test)
regr_predict

array([0.82230403, 0.53142224, 0.81991923, 0.43615962, 0.70473367])

### Avaliação de resultados

In [18]:
metrics.accuracy_score(y_numeric_test, class_predict)
metrics.accuracy_score(y_numeric_test, regr_predict)

0.8

ValueError: Classification metrics can't handle a mix of binary and continuous targets

## Extras

### Baseline classifier

É muito difícil avaliar o resultado de um modelo (descobrir se o valor resultante das métricas é bom ou não), mas existe uma forma de descobrir qual seria o pior valor possível. A ideia é usar um classificador "dummy", isto é, uma estratégia ingênua de classificação. Neste caso apresentado abaixo, o dummy checa qual a classe dominante e classifica as instâncias de teste com aquela mesma classe.

In [ ]:
classification = dummy.DummyClassifier(strategy='most_frequent')
classification.fit(X_nominal_train, y_nominal_train)

In [ ]:
pred = classification.predict(X_nominal_test)

In [ ]:
metrics.accuracy_score(y_nominal_test, pred)